#Overview & motivation 

All of our team members enjoy movies.  In addition to enjoying movies, we also enjoy working with API’s and somewhat structured data sets. Therefore, determining what makes a movie successful using the data available in the Internet Movie Database (IMDB) and Wikipedia seemed like a natural choice. 

In [1]:
from imdb import IMDb
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
import cPickle as pickle
ia = IMDb(accessSystem='http')
from collections import defaultdict 
import io

#Related Work 

#Initial Questions 

#Data 

In [2]:
# Deborah's function "find_movie"
# Will be used to make aadict

def find_movie(title, year,  mlist):
    # find movies that came out in the same year                                                                                                                                    
    year_list = []
    for movie in mlist:
        try:
            if movie.data['year'] == int(year):
                year_list.append(movie)
        except:
            pass
    # if the years do not match, there is no match                                                                                                                                  
    if len(year_list) < 1:
        return None
    else:
        # process the title                                                                                                                                                         
        sorted_title = "".join(sorted(title)).replace(" ", "")
        len_sorted_title = len(sorted_title)
        # check whether movies that came out in the same year                                                                                                                       
        # have the same letters                                                                                                                                                     
        counts = [0]*len(year_list)
        for j in range(len(year_list)):
            mtitle = year_list[j]['title']
            sorted_mtitle = "".join(sorted(mtitle)).replace(" ", "")
            if len_sorted_title == len(sorted_mtitle):
                # if the title cannot be converted to a string                                                                                                                      
                # it is not the correct title                                                                                                                                       
                try:
                    sorted_mtitle = str(sorted_mtitle)
                except:
                    continue
                for i in range(len_sorted_title):
                    if sorted_title[i] == sorted_mtitle[i]:
                        counts[j] += 1
            else:
                return None
        k = counts.index(max(counts))
        if len(year_list) >= 1:
            return year_list[k]
        else:
            return None

Pulling in Oscar CSV (Dana's Work)

In [58]:
# Read in Academy Awards df (AAdf)
AAdf = pd.read_excel("Academy_Awards_2006.xls")

# Update df
# Concat Sort Title with first part of year; store in "title"
# This will be helpful when using ia.search_movie function
AAdf['Year'] = AAdf['Year'].values.astype(str)
AAdf['yr'] = AAdf.apply( lambda row: row['Year'][:4],axis=1 )
AAdf['titleyr'] = AAdf.apply( lambda row: '%s (%s)' % (row['Sort Title'],row['Year'][:4]),axis=1)
# Convert 'Winner' "X" to 1
AAdf['Winner?'] = 1*(AAdf['Winner?'] == 'X')
## TO DO
### Convert 'Country' "0" to USA
# Convert 'Sort Title' to be strings (easier for comparison later)
#AAdf['Sort Title'] = str(AAdf['Sort Title'])
# If movie title is "[no specific film title]", delete
AAdf = AAdf[AAdf['Sort Title'] != "[no specific film title]"].copy()

# Store all possible awards in "awards" list
# This will be used when making aadict to indicate which awards the movie was nominated for/won
awards = list(set(list(AAdf['Category'])))

# Subset the data for easier use when making code
AAsubdf = AAdf[:12]
# View df
AAsubdf.head(5)

,Original Title,English Title,Sort Title,Year,Country,Award,Category,Winner?,Nominee(s),__ Academy Awards,Item,yr,titleyr
0,7th Heaven,7th Heaven,7th Heaven,1927-1928,0,Academy Award,Best Picture,0,Fox,1st,1,1927,7th Heaven (1927)
1,The Racket,The Racket,Racket,1927-1928,0,Academy Award,Best Picture,0,The Caddo Company,1st,2,1927,The Racket (1927)
2,Wings,Wings,Wings,1927-1928,0,Academy Award,Best Picture,1,Paramount Famous Lasky,1st,3,1927,Wings (1927)
3,Chang: A Drama of the Wilderness,Chang,Chang: A Drama of the Wilderness,1927-1928,0,Academy Award,Best Unique and Artistic Picture,0,Paramount Famous Lasky,1st,4,1927,Chang: A Drama of the Wilderness (1927)
4,The Crowd,The Crowd,Crowd,1927-1928,0,Academy Award,Best Unique and Artistic Picture,0,Metro-Goldwyn-Mayer,1st,5,1927,The Crowd (1927)


Note:  You can skip this cell and just run the next cell (pickle.load) to get the complete moviedict

In [104]:
%%time

# Get unique list of movie titles from AAdf
AAuniquemovies = list(set(AAdf['Sort Title']))
# Keep track of failed movies (i.e. movieobj = None)
movieobjNone = list()

AAdict = {}
for i in range(0,len(AAuniquemovies)):
    
    ## Get arguments needed to search the movie title ##
    # movie title
    try:
        int(AAuniquemovies[i])
        arg1 = AAuniquemovies[i]
        arg2 = list(AAdf[AAdf['Sort Title'] == AAuniquemovies[i]]['yr'])[0]  # movie year
        arg3 = ia.search_movie(str(arg1))  # list of possible movies
        titleyr = list(AAdf[AAdf['Sort Title'] == AAuniquemovies[i]]['titleyr'])[0]
        movieobj = ia.search_movie("titleyr")
        if movieobj is None or len(movieobj)>1:
            movieobj = find_movie(str(arg1), arg2, arg3) # find the movie
    except ValueError:
        try:
            arg1 = str(AAuniquemovies[i])
            arg2 = list(AAdf[AAdf['Sort Title'] == AAuniquemovies[i]]['yr'])[0]  # movie year
            arg3 = ia.search_movie(arg1)  # list of possible movies
            titleyr = list(AAdf[AAdf['Sort Title'] == AAuniquemovies[i]]['titleyr'])[0]
            movieobj = ia.search_movie("titleyr")
            if movieobj is None or len(movieobj)>1:
                movieobj = find_movie(arg1, arg2, arg3) # find the movie
        except ValueError:
            arg1 = AAuniquemovies[i]
            arg2 = list(AAdf[AAdf['Sort Title'] == AAuniquemovies[i]]['yr'])[0]  # movie year
            arg3 = ia.search_movie(arg1)  # list of possible movies
            titleyr = list(AAdf[AAdf['Sort Title'] == AAuniquemovies[i]]['titleyr'])[0]
            movieobj = ia.search_movie("titleyr")
            if movieobj is None or len(movieobj)>1:
                movieobj = find_movie(arg1, arg2, arg3) # find the movie
    # Could be that movie year is 1 year off (due to the Oscars being biennial in the beginning)
    if movieobj is None:
        arg2 = int(arg2) + 1
        movieobj = find_movie(arg1, arg2, arg3)
        if (movieobj is None) & (arg3 != []):
            movieobj = arg3[0]
    print i, movieobj
    
    # Had to add this because some movies weren't getting any movies back ("Mighty Mouse in Gypsy Life" #247)
    if movieobj is not None:
        ## Get movie id ##
        movid = movieobj.movieID
        ## Populate dictionary, main key is movie id ##
        AAdict[movid] = {}
        # "title": title of movie
        AAdict[movid]['title'] = list(AAdf[AAdf['Sort Title']==arg1]['Sort Title'])[0]
        # "nominations": list of Oscar nominations
        AAdict[movid]['nominations'] = list(AAdf[AAdf['Sort Title']==arg1]['Category'])
        # "won": list of Oscars won
        AAdict[movid]['won'] = list(AAdf[(AAdf['Sort Title']==arg1) & (AAdf['Winner?']==1)]['Category'])
        # "year": year Oscar won
        AAdict[movid]['year'] = list(AAdf[AAdf['Sort Title']==arg1]['Year'])[0]
        # "country": country of movie
        AAdict[movid]['country'] = list(AAdf[AAdf['Sort Title']==arg1]['Country'])[0]
        # make each award individual key and the value to indicate whether movie won/nominated or not
        # Loop through awards list and indicate if movie was nominated or won
        for award in awards:
            # "Nominated award_name": True or False
            AAdict[movid]["Nominated %s" % award] = award in list(AAdict[movid]['nominations'])
            # "Nominated award_name": True or False
            AAdict[movid]["Won %s" % award] = award in list(AAdict[movid]['won'])
        ## TO DO, list person/company associated with the award!!
    else:
        movieobjNone.append((i, AAuniquemovies[i]))

0 Addams Family Values
1 Ernst & lyset
2 Complaints of a Dutiful Daughter
3 Separate Tables
4 Accidental Tourist
5 9 (I)
6 Breaking Away
7 Blackboard Jungle
8 Rob Roy
9 Mike's New Car
10 Creature Comforts
11 Double or Nothing
12 So You Think You're Not Guilty
13 Man of a Thousand Faces
14 Valmont
15 12:01 PM
16 Under Siege
17 Redwoods
18 Day for Night
19 Hondo
20 Stolen Life
21 The Sound of Music
22 Ice-Capades
23 Monsieur Vincent
24 Once More, My Darling
25 Wait Until Dark
26 Big Sky
27 Parenthood
28 Every Day's a Holiday
29 Alice Adams
30 Liberators: Fighting on Two Fronts in World War II
31 Lorenzo's Oil
32 Flying Down to Rio
33 Make a Wish
34 10
35 Ghostbusters
36 When Dinosaurs Ruled the Earth
37 The First Lessons in Love (1978) (TV Episode) - People Like Us
38 Is Paris Burning?
39 Johann Mouse
40 Kangaroo Court
41 Irma la Douce
42 It's a Mad, Mad, Mad, Mad World
43 Battle Cry
44 Walky Talky Hawky
45 Night and Day
46 Here Comes the Band
47 The Ra Expeditions
48 Volcano: An Inquiry

In [146]:
# CHECK
print "number of movies in AAdict:", len(AAdict.keys()) + len(movieobjNone)
print "number of movies in AAdf:", len(AAuniquemovies)
    # inconsistent numbers...
    
## DEBUG ##
# Get list of what is in the dictionary & what movies returned "None"
debuglist = list()
for i in range(0,len(list(AAdict.keys()))):
    movid = list(AAdict.keys())[i]
    debuglist.append(AAdict[movid]['title'])
for i in range(0,len(movieobjNone)):
    debuglist.append(movieobjNone[i][1])    
# Compare to list of movies from AAdf
print "list of movies missing from AAdict:"
list(set(AAuniquemovies) - set(debuglist))

number of movies in AAdict: 4094
number of movies in AAdf: 4119
list of movies missing from AAdict:


[u'Cet obscur objet du d\xe9sir',
 u'Wargames',
 u'Kitty Foyle',
 u'Wag the Dog',
 u'Da hong deng long gao gao gua',
 u'Seven Beauties',
 u'Rashomon',
 u'Shoeshine',
 u'Ma nuit chez Maud',
 u'Pelle erobreren',
 u'Star Trek: Motion Picture',
 u'Little Matchgirl',
 u'Slipper and the Rose',
 u'Il Casanova di Federico Fellini',
 u'La Cage Aux Folles',
 u"No Man's Land",
 u'Caravan',
 u'Lust for Life',
 u'Indagine su un cittadino al di sopra di ogni sospetto',
 u'Greystoke: The Legend of Tarzan, Lord of the Apes',
 u'Enemies: A Love Story',
 u'Triplets of Belleville (Les Triplettes de Belleville)',
 u'Gold Diggers of 1935',
 u'Place in the Sun',
 u'Blue Veil']

In [164]:
#x = ia.search_movie("No Man's Land")
#print find_movie("No Man's Land", 2001, x)
#AAdf[AAdf['Sort Title'] == "No Man's Land"]
AAuniquemovies.index("No Man's Land")
AAuniquemovies[756]

u"No Man's Land"

In [213]:
i = 756
arg1 = str(AAuniquemovies[i])
arg2 = list(AAdf[AAdf['Sort Title'] == AAuniquemovies[i]]['yr'])[0]  # movie year
arg3 = ia.search_movie("No Man's Land (2001)")  # list of possible movies
titleyr = list(AAdf[AAdf['Sort Title'] == AAuniquemovies[i]]['titleyr'])[0]
movieobj = ia.search_movie("titleyr")
if movieobj is None or len(movieobj)>1:
    movieobj = find_movie(arg1, arg2, arg3) # find the movie
    print 1
print arg1
print arg2
print titleyr
print movieobj

1
No Man's Land
2001
No Man's Land (2001)
None


In [214]:
x = ia.search_movie("No Man's Land (2001)")
find_movie("No Man's Land", 2001, ia.search_movie("No Man's Land (2001)"))

In [224]:
year_list = []
arg3 = ia.search_movie("No Man's Land (2001)")
year = 2001
for movie in arg3:
    print movie.data['year']
    try:
        if movie.data['year'] == int(year):
            year_list.append(movie)
    except:
        pass
print year_list
# if the years do not match, there is no match                                                                                                                                  
if len(year_list) < 1:
    print None
else:
    # process the title                                                                                                                                                         
    sorted_title = "".join(sorted(title)).replace(" ", "")
    len_sorted_title = len(sorted_title)
    # check whether movies that came out in the same year                                                                                                                       
    # have the same letters                                                                                                                                                     
    counts = [0]*len(year_list)
    for j in range(len(year_list)):
        mtitle = year_list[j]['title']
        sorted_mtitle = "".join(sorted(mtitle)).replace(" ", "")
        if len_sorted_title == len(sorted_mtitle):
            # if the title cannot be converted to a string                                                                                                                      
            # it is not the correct title                                                                                                                                       
            try:
                sorted_mtitle = str(sorted_mtitle)
            except:
                continue
            for i in range(len_sorted_title):
                if sorted_title[i] == sorted_mtitle[i]:
                    counts[j] += 1
        else:
            print "None2"   

2001
2001
2001
2009
2001
[<Movie id:0283509[http] title:_No Man's Land (I) (2001)_>, <Movie id:3425390[http] title:_No Man's Land (II) (2001)_>, <Movie id:1267587[http] title:_"Dans une galaxie près de chez vous" No Man's Land (2001)_>, <Movie id:0141907[http] title:_Town & Country (2001)_>]
Town & Country
&CTnnoortuwy


In [ ]:
# find movies that came out in the same year                                                                                                                                    
year_list = []
for movie in mlist:
    try:
        if movie.data['year'] == int(year):
            year_list.append(movie)
    except:
        print "here"
# if the years do not match, there is no match                                                                                                                                  
if len(year_list) < 1:
    print None
else:
    # process the title                                                                                                                                                         
    sorted_title = "".join(sorted(str(title))).replace(" ", "")
    len_sorted_title = len(sorted_title)
    # check whether movies that came out in the same year                                                                                                                       
    # have the same letters                                                                                                                                                     
    counts = [0]*len(year_list)
    for j in range(len(year_list)):
        mtitle = year_list[j]['title']
        sorted_mtitle = "".join(sorted(mtitle)).replace(" ", "")
        if len_sorted_title == len(sorted_mtitle):
            # if the title cannot be converted to a string                                                                                                                      
            # it is not the correct title                                                                                                                                       
            try:
                sorted_mtitle = str(sorted_mtitle)
            except:
                continue
            for i in range(len_sorted_title):
                if sorted_title[i] == sorted_mtitle[i]:
                    counts[j] += 1
        else:
            print None
    k = counts.index(max(counts))
    if len(year_list) >= 1:
        print year_list[k]
    else:
        print None

In [112]:
pickle.dump(AAdict,open('moviedict','wb'))
#AAdict = pickle.load(open('moviedict','rb'))

For easier use in calling movies, change moviedict keys to be movieID
Changes from above:
1)  rename moviedict to aadict for "academy award dictionary"

In [11]:
filename = 'moviestemp.p'
movies = pickle.load(io.open(filename,'rb'))

In [222]:
a = ia.get_movie('5072808')
b = ia.get_movie('5072808')

In [30]:
a.isSameMovie(b)

1

In [10]:
movies

[<Movie id:5072808[http] title:_0 Degree C: A Love Story (2009)_>,
 <Movie id:4494824[http] title:_0 Feet Away (2014)_>,
 <Movie id:0244940[http] title:_0 Uhr 15, Zimmer 9 (1950)_>,
 <Movie id:3310038[http] title:_0+ omikron plus (2013)_>,
 <Movie id:1789663[http] title:_0,5 KM2 North (2010)_>,
 <Movie id:3692700[http] title:_0,60 mg (2014)_>,
 <Movie id:1042462[http] title:_0,7 ya! (2005)_>,
 <Movie id:2081142[http] title:_0.0270270 (2006)_>,
 <Movie id:2416424[http] title:_00 Schneider - Im Wendekreis der Eidechse (2013)_>,
 <Movie id:0109000[http] title:_00 Schneider - Jagd auf Nihil Baxter (1994)_>,
 <Movie id:4029848[http] title:_00-Baby Boom Child (2014)_>,
 <Movie id:0215459[http] title:_00/ciak operazione mondo (1966)_>,
 <Movie id:0057805[http] title:_002 operazione Luna (1965)_>,
 <Movie id:2183050[http] title:_003064.XZ (2012)_>,
 <Movie id:2141567[http] title:_006. un principio (2011)_>,
 <Movie id:5031210[http] title:_0068 Sniper's Nest (2015)_>,
 <Movie id:2158351[http] t

In [20]:
## If I get AAdict working, this can go
x = AAdf
moviedict = dict.fromkeys(x['Original Title'])
for movie in moviedict:
    moviedict[movie] = {}
    moviedict[movie]['nominations'] = list(x[x['Original Title']==movie]['Category'])
    moviedict[movie]['won'] = list(x[(x['Original Title']==movie) & (x['Winner?']==1)]['Category'])
    moviedict[movie]['titleyr'] = list(x[x['Original Title']==movie]['titleyr'])[0]
    moviedict[movie]['year'] = list(x[x['Original Title']==movie]['Year'])[0]
    moviedict[movie]['nominees'] = list(x[x['Original Title']==movie]['Nominee(s)'])
    moviedict[movie]['titles'] = ia.search_movie(list(x[x['Original Title']==movie]['titleyr'])[0], _episodes=False)
    # For complete listing of movies...
    # ** Currently assuming the first movie in list is the correct one! **
    movieids = list()
    for i in range(0,len(moviedict[movie]['titles'])):
        movieids.append(moviedict[movie]['titles'][i].movieID)
    moviedict[movie]['movieIDs'] = movieids
    # Loop through awards list and indicate if movie was nominated or won
    for award in awards:
        moviedict[movie]["Nominated %s" % award] = award in list(moviedict[movie]['nominations'])
        moviedict[movie]["Won %s" % award] = award in list(moviedict[movie]['won'])
    # Only enter this loop if there was an imdb movieid found
    #if len(movieids) > 0:
        #mov = ia.get_movie(movieids[0])
        #moviedict[movie]['prodcomp'] = mov.data['production companies']
        #moviedict[movie]['cast'] = mov.data['cast']

In [16]:
%time

# Get unique list of movie titles from AAdf
AAuniquemovies = list(set(AAdf['Sort Title']))

AAdict = {}
for i in range(0,len(AAuniquemovies)):
    
    ## Get arguments needed to search the movie title ##
    # movie title
    try:
        arg1 = AAuniquemovies[i]
        arg2 = list(AAdf[AAdf['Sort Title'] == AAuniquemovies[i]]['yr'])[0]  # movie year
        arg3 = ia.search_movie(str(arg1))  # list of possible movies
        movieobj = find_movie(str(arg1), arg2, arg3) # find the movie
    except ValueError:
        try:
            arg1 = str(AAuniquemovies[i])
            arg2 = list(AAdf[AAdf['Sort Title'] == AAuniquemovies[i]]['yr'])[0]  # movie year
            arg3 = ia.search_movie(arg1)  # list of possible movies
            movieobj = find_movie(arg1, arg2, arg3) # find the movie
        except ValueError:
            arg1 = AAuniquemovies[i]
            arg2 = list(AAdf[AAdf['Sort Title'] == AAuniquemovies[i]]['yr'])[0]  # movie year
            arg3 = ia.search_movie(arg1)  # list of possible movies
            movieobj = find_movie(arg1, arg2, arg3) # find the movie
    # Could be that movie year is 1 year off (due to the Oscars being biennial in the beginning)
    if movieobj is None:
        arg2 = int(arg2) + 1
        movieobj = find_movie(arg1, arg2, arg3)
        if (movieobj is None) & (arg3 != []):
            movieobj = arg3[0]
    #print i, movieobj
    
    # Had to add this because some movies weren't getting any movies back ("Mighty Mouse in Gypsy Life" #247)
    if movieobj is not None:
        ## Get movie id ##
        movid = movieobj.movieID
        ## Populate dictionary, main key is movie id ##
        AAdict[movid] = {}
        # "title": title of movie
        AAdict[movid]['title'] = list(AAdf[AAdf['Sort Title']==arg1]['Sort Title'])[0]
        # "nominations": list of Oscar nominations
        AAdict[movid]['nominations'] = list(AAdf[AAdf['Sort Title']==arg1]['Category'])
        # "won": list of Oscars won
        AAdict[movid]['won'] = list(AAdf[(AAdf['Sort Title']==arg1) & (AAdf['Winner?']==1)]['Category'])
        # "year": year Oscar won
        AAdict[movid]['year'] = list(AAdf[AAdf['Sort Title']==arg1]['Year'])[0]
        # "country": country of movie
        AAdict[movid]['country'] = list(AAdf[AAdf['Sort Title']==arg1]['Country'])[0]
        # make each award individual key and the value to indicate whether movie won/nominated or not
        # Loop through awards list and indicate if movie was nominated or won
        for award in awards:
            # "Nominated award_name": True or False
            AAdict[movid]["Nominated %s" % award] = award in list(AAdict[movid]['nominations'])
            # "Nominated award_name": True or False
            AAdict[movid]["Won %s" % award] = award in list(AAdict[movid]['won'])
        ## TO DO, list person/company associated with the award!!

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


TypeError: 'int' object is not iterable

#Exploratory Data Analysis 

#Final Analysis

#Presentation 